In [1]:
# Cài đặt thư viện Playwright
!pip install playwright

# Cài đặt các trình duyệt (Chromium, Firefox, Webkit)
!playwright install

# Cài đặt các thư viện hệ thống cần thiết cho Playwright
!playwright install-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.3 MB/s eta 0:00:00
(node:775) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
167.3 MiB [] 0% 0.0s167.3 MiB [] 0% 41.4s167.3 MiB [] 0% 18.0s167.3 MiB [] 0% 15.3s167.3 MiB [] 0% 8.4s167.3 MiB [] 1% 5.0s167.3 MiB [] 2% 4.1s167.3 MiB [] 3% 3.3s167.3 MiB [] 4% 3.2s167.3 MiB [] 5% 2.8s167.3 MiB [] 6% 2.5s167.3 MiB [] 7% 2.4s167.3 MiB [] 8% 2.3s167.3 MiB [] 9% 2.2s167.3 MiB [] 10% 2.2s167.3 MiB [] 11% 2.0s167.3 MiB [] 12% 2.0s167.3 MiB [] 13% 1.9s167.3 MiB [] 14% 1.9s167.3 MiB [] 15% 1.9s167.3 MiB [] 16% 1.8s167.3 MiB [] 17% 1.8s167.3 MiB [] 18% 1.8s167.3 MiB [] 19% 1.8s167.3 MiB [] 20% 1.8s167.3 MiB [] 21% 1.7s167.3 MiB [] 22% 1.6s167.3 MiB [] 24% 1.6s167.3 MiB [] 25% 1.5s167.3 MiB [] 26% 1.5s167

In [2]:
import sys
import random
import asyncio
import os
from playwright.async_api import async_playwright
import json

output_file = "scholar_articles.json"

In [3]:
def load_existing_data(output_file):
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

# Tài chính

In [4]:
async def run():
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=[
                "--no-sandbox",
                "--disable-dev-shm-usage"
            ]
        )

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        )

        page = await context.new_page()

        data_list = load_existing_data(output_file)
        max_articles = 110

        base_url = "https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+tài+chính"

        os.makedirs("pdf_files", exist_ok=True)

        for start in range(0, max_articles, 10):

            url = f"{base_url}&start={start}"
            print(f"\nĐang truy cập: {url}")

            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_selector("div.gs_ri", timeout=10000)
            except:
                print("Có thể bị CAPTCHA")
                await asyncio.sleep(5)
                continue

            articles = page.locator("div.gs_r.gs_or.gs_scl")

            count = await articles.count()
            print(f"→ Tìm thấy {count} bài")

            for i in range(count):

                if len(data_list) >= max_articles:
                    break

                article = articles.nth(i)

                title = await article.locator("h3.gs_rt").inner_text()

                # ----------- LẤY LINK PDF BÊN PHẢI ------------
                pdf_link = None
                pdf_block = article.locator("div.gs_or_ggsm a")

                if await pdf_block.count() > 0:
                    pdf_link = await pdf_block.get_attribute("href")

                print(f"\n--- Bài {len(data_list)+1} ---")
                print("Tiêu đề:", title)
                print("PDF:", pdf_link)

                # ----------- TẢI PDF ------------
                if pdf_link:
                    try:
                        response = await context.request.get(pdf_link)
                        if response.status == 200:
                            file_path = f"pdf_files/paper_{len(data_list)+1}.pdf"
                            with open(file_path, "wb") as f:
                                f.write(await response.body())
                            print("Đã tải:", file_path)
                        else:
                            print("Không tải được (status != 200)")
                    except Exception as e:
                        print("Lỗi tải PDF:", e)

                data = {
                    "source": "Google Scholar",
                    "topic": "Bài báo tài chính",
                    "article_title": title,
                    "pdf_url": pdf_link
                }

                data_list.append(data)

                await asyncio.sleep(random.uniform(2, 4))

            await asyncio.sleep(random.uniform(4, 8))

        print(f"\nĐÃ LẤY TỔNG CỘNG: {len(data_list)} BÀI")
        await browser.close()

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data_list, f, ensure_ascii=False, indent=4)

        print(f"Đã lưu dữ liệu vào {output_file}")


await run()


Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+tài+chính&start=0
→ Tìm thấy 10 bài

--- Bài 1 ---
Tiêu đề: [PDF] Tích hợp Toán học và giáo dục tài chính trong dạy học thiết lập kế hoạch đầu tư cá nhân ở lớp 10
PDF: http://vjes.vnies.edu.vn/sites/default/files/khgdvn_-_tap_19_-_so_s3_-61-67.pdf
Đã tải: pdf_files/paper_1.pdf

--- Bài 2 ---
Tiêu đề: Phân tích báo cáo tài chính
PDF: https://thuvien.kgc.edu.vn/bitstream/123456789/1982/1/36.MD36.%20Phan%20tich%20bao%20cao%20tai%20chinh.pdf
Đã tải: pdf_files/paper_2.pdf

--- Bài 3 ---
Tiêu đề: Nghiên cứu sự ảnh hưởng của cấu trúc tài chính và rủi ro tài chính tới hiệu quả kinh doanh các doanh nghiệp niêm yết trên thị trường chứng khoán Việt Nam
PDF: https://js.ktpt.edu.vn/index.php/jed/article/download/373/165
Lỗi tải PDF: APIRequestContext.get: unable to verify the first certificate; if the root CA is installed locally, try running Node.js with --use-system-ca
Call log:
  - → GET https://js.ktpt.edu.vn/index.php/jed/ar

# Nghệ thuật

In [5]:
async def run():
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=[
                "--no-sandbox",
                "--disable-dev-shm-usage"
            ]
        )

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        )

        page = await context.new_page()

        data_list = load_existing_data(output_file)
        max_articles = 110

        base_url = "https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+nghệ+thuật"

        os.makedirs("pdf_files", exist_ok=True)

        for start in range(0, max_articles, 10):

            url = f"{base_url}&start={start}"
            print(f"\nĐang truy cập: {url}")

            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_selector("div.gs_ri", timeout=10000)
            except:
                print("Có thể bị CAPTCHA")
                await asyncio.sleep(5)
                continue

            articles = page.locator("div.gs_r.gs_or.gs_scl")

            count = await articles.count()
            print(f"→ Tìm thấy {count} bài")

            for i in range(count):

                if len(data_list) >= max_articles:
                    break

                article = articles.nth(i)

                title = await article.locator("h3.gs_rt").inner_text()

                # ----------- LẤY LINK PDF BÊN PHẢI ------------
                pdf_link = None
                pdf_block = article.locator("div.gs_or_ggsm a")

                if await pdf_block.count() > 0:
                    pdf_link = await pdf_block.get_attribute("href")

                print(f"\n--- Bài {len(data_list)+1} ---")
                print("Tiêu đề:", title)
                print("PDF:", pdf_link)

                # ----------- TẢI PDF ------------
                if pdf_link:
                    try:
                        response = await context.request.get(pdf_link)
                        if response.status == 200:
                            file_path = f"pdf_files/paper_{len(data_list)+1}.pdf"
                            with open(file_path, "wb") as f:
                                f.write(await response.body())
                            print("Đã tải:", file_path)
                        else:
                            print("Không tải được (status != 200)")
                    except Exception as e:
                        print("Lỗi tải PDF:", e)

                data = {
                    "source": "Google Scholar",
                    "topic": "Bài báo nghệ thuật",
                    "article_title": title,
                    "pdf_url": pdf_link
                }

                data_list.append(data)

                await asyncio.sleep(random.uniform(2, 4))

            await asyncio.sleep(random.uniform(4, 8))

        print(f"\nĐÃ LẤY TỔNG CỘNG: {len(data_list)} BÀI")
        await browser.close()

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data_list, f, ensure_ascii=False, indent=4)

        print(f"Đã lưu dữ liệu vào {output_file}")


await run()


Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+nghệ+thuật&start=0
→ Tìm thấy 10 bài

--- Bài 101 ---
Tiêu đề: [HTML] Vấn Nạn Nghệ Thuật Trong Giảng Trình: Cội Nguồn Tác Phẩm Nghệ Thuật Của Martin Heidegger: The Problem of Art in the Lecture Course: The Origin of the …
PDF: https://khoahocconggiao.org/index.php/tckh/article/view/176
Đã tải: pdf_files/paper_101.pdf

--- Bài 102 ---
Tiêu đề: [PDF] … SÁT THỜI GIAN NGH I GIAN NGH I GIAN NGHỆ THUẬT GẮN VỚI CÁC PHƯƠNG TH I CÁC PHƯƠNG TH I CÁC PHƯƠNG THỨC DỰ BÁO TRONG VĂN XUÔI …
PDF: https://hnmu.edu.vn/upload/user/tin-bai/tap-chi/tckh-so-12-kh-xa-hoi-va-giao-duc-dh-thu-do-02-2017.pdf#page=70
Đã tải: pdf_files/paper_102.pdf

--- Bài 103 ---
Tiêu đề: [PDF] Nghệ thuật xây dựng nhân vật trong loại hình tự sự văn học hiện đại dân tộc Thái
PDF: https://scholar.dlu.edu.vn/thuvienso/bitstream/DLU123456789/173588/1/CVv468V8S32022377.pdf
Lỗi tải PDF: APIRequestContext.get: unable to verify the first certificate; if the root CA is

# Chính trị

In [7]:
async def run():
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=[
                "--no-sandbox",
                "--disable-dev-shm-usage"
            ]
        )

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        )

        page = await context.new_page()

        data_list = load_existing_data(output_file)
        max_articles = 110

        base_url = "https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị"

        os.makedirs("pdf_files", exist_ok=True)

        for start in range(0, max_articles, 10):

            url = f"{base_url}&start={start}"
            print(f"\nĐang truy cập: {url}")

            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_selector("div.gs_ri", timeout=10000)
            except:
                print("Có thể bị CAPTCHA")
                await asyncio.sleep(5)
                continue

            articles = page.locator("div.gs_r.gs_or.gs_scl")

            count = await articles.count()
            print(f"→ Tìm thấy {count} bài")

            for i in range(count):

                if len(data_list) >= max_articles:
                    break

                article = articles.nth(i)

                title = await article.locator("h3.gs_rt").inner_text()

                # ----------- LẤY LINK PDF BÊN PHẢI ------------
                pdf_link = None
                pdf_block = article.locator("div.gs_or_ggsm a")

                if await pdf_block.count() > 0:
                    pdf_link = await pdf_block.get_attribute("href")

                print(f"\n--- Bài {len(data_list)+1} ---")
                print("Tiêu đề:", title)
                print("PDF:", pdf_link)

                # ----------- TẢI PDF ------------
                if pdf_link:
                    try:
                        response = await context.request.get(pdf_link)
                        if response.status == 200:
                            file_path = f"pdf_files/paper_{len(data_list)+1}.pdf"
                            with open(file_path, "wb") as f:
                                f.write(await response.body())
                            print("Đã tải:", file_path)
                        else:
                            print("Không tải được (status != 200)")
                    except Exception as e:
                        print("Lỗi tải PDF:", e)

                data = {
                    "source": "Google Scholar",
                    "topic": "Bài báo chính trị",
                    "article_title": title,
                    "pdf_url": pdf_link
                }

                data_list.append(data)

                await asyncio.sleep(random.uniform(2, 4))

            await asyncio.sleep(random.uniform(4, 8))

        print(f"\nĐÃ LẤY TỔNG CỘNG: {len(data_list)} BÀI")
        await browser.close()

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data_list, f, ensure_ascii=False, indent=4)

        print(f"Đã lưu dữ liệu vào {output_file}")


await run()


Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=0
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=10
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=20
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=30
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=40
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=50
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=60
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=70
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=bài+báo+chính+trị&start=80
→ Tìm thấy 10 bài

Đang truy cập: https://scholar.google

In [8]:
!pip install requests bs4 lxml

In [9]:
import requests
from bs4 import BeautifulSoup
import urllib3
import json
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [10]:
class VJST_Scraper:

    def __init__(self, base_url, topic):
        self.base_url = base_url
        self.topic = topic
        self.headers = {
            "User-Agent": "Mozilla/5.0"
        }
        self.articles = []

    def crawl(self, max_pages=20):

        for page in range(1, max_pages + 1):

            if page == 1:
                url = self.base_url
            else:
                url = f"{self.base_url}/{page}"

            print(f"\n--- Đang crawl {self.topic} | Trang {page} ---")

            try:
                res = requests.get(url, headers=self.headers, verify=False, timeout=15)
                soup = BeautifulSoup(res.text, "lxml")

                summaries = soup.select(".obj_article_summary")

                if not summaries:
                    print("Không còn bài.")
                    break

                for s in summaries:

                    title_tag = s.select_one(".title a")
                    pdf_tag = s.select_one("a.obj_galley_link.pdf")

                    if title_tag and pdf_tag:

                        title = title_tag.get_text(strip=True)
                        pdf_url = pdf_tag["href"].replace("/view/", "/download/")

                        self.articles.append({
                            "source": "Tạp chí Khoa học và Công nghệ Việt Nam",
                            "topic": self.topic,
                            "title": title,
                            "pdf_url": pdf_url
                        })

                time.sleep(0.5)

            except Exception as e:
                print("Lỗi:", e)
                break

        print(f"✓ {self.topic}: Lấy được {len(self.articles)} bài.")

In [11]:
all_articles = []
seen_pdf = set()  # để loại trùng

# Khoa học - Y dược

In [12]:
scraper = VJST_Scraper("https://b.vjst.vn/index.php/ban_b/section/view/khoa-hoc-y-duoc", "Khoa học y dược")
scraper.crawl(max_pages=20)

for article in scraper.articles:
    if article["pdf_url"] not in seen_pdf:
        all_articles.append(article)
        seen_pdf.add(article["pdf_url"])


--- Đang crawl Khoa học y dược | Trang 1 ---

--- Đang crawl Khoa học y dược | Trang 2 ---

--- Đang crawl Khoa học y dược | Trang 3 ---

--- Đang crawl Khoa học y dược | Trang 4 ---

--- Đang crawl Khoa học y dược | Trang 5 ---

--- Đang crawl Khoa học y dược | Trang 6 ---

--- Đang crawl Khoa học y dược | Trang 7 ---

--- Đang crawl Khoa học y dược | Trang 8 ---

--- Đang crawl Khoa học y dược | Trang 9 ---

--- Đang crawl Khoa học y dược | Trang 10 ---

--- Đang crawl Khoa học y dược | Trang 11 ---

--- Đang crawl Khoa học y dược | Trang 12 ---

--- Đang crawl Khoa học y dược | Trang 13 ---

--- Đang crawl Khoa học y dược | Trang 14 ---

--- Đang crawl Khoa học y dược | Trang 15 ---

--- Đang crawl Khoa học y dược | Trang 16 ---

--- Đang crawl Khoa học y dược | Trang 17 ---

--- Đang crawl Khoa học y dược | Trang 18 ---

--- Đang crawl Khoa học y dược | Trang 19 ---

--- Đang crawl Khoa học y dược | Trang 20 ---
✓ Khoa học y dược: Lấy được 199 bài.


# Khoa học kỹ thuật và công nghệ

In [13]:
scraper = VJST_Scraper("https://b.vjst.vn/index.php/ban_b/section/view/khoa-hoc-ky-thuat-va-cong-nghe", "Khoa học kỹ thuật và công nghệ")
scraper.crawl(max_pages=20)

for article in scraper.articles:
    if article["pdf_url"] not in seen_pdf:
        all_articles.append(article)
        seen_pdf.add(article["pdf_url"])


--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 1 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 2 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 3 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 4 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 5 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 6 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 7 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 8 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 9 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 10 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 11 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 12 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 13 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 14 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 15 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 16 ---

--- Đang crawl K

# Khoa học Nông nghiệp

In [14]:
scraper = VJST_Scraper("https://b.vjst.vn/index.php/ban_b/section/view/khoa-hoc-nong-nghiep", "Khoa học kỹ thuật và công nghệ")
scraper.crawl(max_pages=50)

for article in scraper.articles:
    if article["pdf_url"] not in seen_pdf:
        all_articles.append(article)
        seen_pdf.add(article["pdf_url"])


--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 1 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 2 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 3 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 4 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 5 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 6 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 7 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 8 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 9 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 10 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 11 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 12 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 13 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 14 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 15 ---

--- Đang crawl Khoa học kỹ thuật và công nghệ | Trang 16 ---

--- Đang crawl K

# Khoa học xã hội và nhân văn

In [15]:
scraper = VJST_Scraper("https://b.vjst.vn/index.php/ban_b/section/view/khoa-hoc-xa-hoi-va-nhan-van", "Khoa học xã hội và nhân văn")
scraper.crawl(max_pages=50)

for article in scraper.articles:
    if article["pdf_url"] not in seen_pdf:
        all_articles.append(article)
        seen_pdf.add(article["pdf_url"])


--- Đang crawl Khoa học xã hội và nhân văn | Trang 1 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 2 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 3 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 4 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 5 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 6 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 7 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 8 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 9 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 10 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 11 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 12 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 13 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 14 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 15 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 16 ---

--- Đang crawl Khoa học xã hội và nhân văn | Trang 17 ---

--- Đ

# Khoa học tự nhiên

In [16]:
scraper = VJST_Scraper("https://b.vjst.vn/index.php/ban_b/section/view/khoa-hoc-tu-nhien", "Khoa học tự nhiên")
scraper.crawl(max_pages=20)

for article in scraper.articles:
    if article["pdf_url"] not in seen_pdf:
        all_articles.append(article)
        seen_pdf.add(article["pdf_url"])


--- Đang crawl Khoa học tự nhiên | Trang 1 ---

--- Đang crawl Khoa học tự nhiên | Trang 2 ---

--- Đang crawl Khoa học tự nhiên | Trang 3 ---

--- Đang crawl Khoa học tự nhiên | Trang 4 ---

--- Đang crawl Khoa học tự nhiên | Trang 5 ---

--- Đang crawl Khoa học tự nhiên | Trang 6 ---

--- Đang crawl Khoa học tự nhiên | Trang 7 ---

--- Đang crawl Khoa học tự nhiên | Trang 8 ---

--- Đang crawl Khoa học tự nhiên | Trang 9 ---

--- Đang crawl Khoa học tự nhiên | Trang 10 ---

--- Đang crawl Khoa học tự nhiên | Trang 11 ---

--- Đang crawl Khoa học tự nhiên | Trang 12 ---

--- Đang crawl Khoa học tự nhiên | Trang 13 ---

--- Đang crawl Khoa học tự nhiên | Trang 14 ---

--- Đang crawl Khoa học tự nhiên | Trang 15 ---

--- Đang crawl Khoa học tự nhiên | Trang 16 ---

--- Đang crawl Khoa học tự nhiên | Trang 17 ---

--- Đang crawl Khoa học tự nhiên | Trang 18 ---

--- Đang crawl Khoa học tự nhiên | Trang 19 ---

--- Đang crawl Khoa học tự nhiên | Trang 20 ---
✓ Khoa học tự nhiên: Lấy được

In [17]:
output_file = "vjst_articles.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)

print("\n=====================================")
print(f"TỔNG CỘNG LƯU {len(all_articles)} BÀI")
print(f"File: {output_file}")
print("=====================================")


TỔNG CỘNG LƯU 995 BÀI
File: vjst_articles.json
